#Na początku instaluję i importuję potrzebne pakiety

In [ ]:
!pip install transformers
!pip install numpy
!pip install pandas
!pip install scipy
!pip install tqdm

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
from tqdm import tqdm
import pandas as pd
tqdm.pandas()

     |████████████████████████████████| 2.1MB 9.8MB/s 
     |████████████████████████████████| 901kB 30.2MB/s 
     |████████████████████████████████| 3.3MB 36.1MB/s 


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


#Teraz wybieram interesujący mnie model.
####Dostępne taski: 
emoji, emotion, hate, irony, offensive, sentiment

In [ ]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:
# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]


In [ ]:

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

Tworzę też funkcje ułatwiające tworzenie predykcji i jej wyświetlanie

In [ ]:
def predict_text(text):
  encoded_input = tokenizer(text, return_tensors='pt', max_length=512, truncation=True)
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  return scores

def print_scores(scores):
  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  for i in range(scores.shape[0]):
      l = labels[ranking[i]]
      s = scores[ranking[i]]
      print(f"{i+1}) {l} {np.round(float(s), 4)}")


# Właściwa analiza sentymentu

W folderze roboczym powinny znaleść pliki: 'ceo_guardian_df.csv', 'cyb_guardian_df.csv', 'env_guardian_df.csv', 'gov_guardian_df.csv'

In [ ]:
def calculate_scores_for_df(filename):
  df = pd.read_csv(filename, sep=',', error_bad_lines=False)

  df['cleaned_text'] = df.apply(lambda row: f'{row.text} - {str(row.storyText)}', axis=1)
  del df['text']
  del df['storyText']
  df = df.drop_duplicates(subset='cleaned_text', keep="first")
  df[
    ['negative_score', 'neutral_score', 'positive_score']
  ] = df['cleaned_text'].progress_apply(lambda review: pd.Series(predict_text(review)))

  df.to_csv(f'sentiment_{filename}', sep=';')

In [ ]:
for filename in ['ceo_guardian_df.csv', 'cyb_guardian_df.csv', 'env_guardian_df.csv', 'gov_guardian_df.csv']:
  calculate_scores_for_df(filename=filename)

100%|██████████| 12/12 [00:22<00:00,  1.87s/it]
